# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Import test and train data  
2. Import the labels ( train and test) 
3. Get the word index and then Create key value pair for word and word_id. (12.5 points)
4. Build a Sequential Model using Keras for Sentiment Classification task. (10 points)
5. Report the Accuracy of the model. (5 points)  
6. Retrive the output of each layer in keras for a given single test sample from the trained model you built. (2.5 points)


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/LabExternal9')

In [3]:
#from sklearn.utils import shuffle
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding
from keras.layers import LSTM

Using TensorFlow backend.


#### Usage:

In [4]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
#(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

# restore np.load for future normal usage
np.load = np_load_old

17465344/17464789 [==============================] - 1s 0us/step


In [5]:
print("x_train ", x_train.shape)
print("y_train ", y_train.shape)
print("_"*100)
print("x_test ", x_test.shape)
print("y_test ", y_test.shape)
print("_"*100)
print("Maximum value of a word index ")
print(max([max(sequence) for sequence in x_train]))
print("Maximum length num words of review in train ")
print(max([len(sequence) for sequence in x_train]))

x_train  (25000,)
y_train  (25000,)
____________________________________________________________________________________________________
x_test  (25000,)
y_test  (25000,)
____________________________________________________________________________________________________
Maximum value of a word index 
9999
Maximum length num words of review in train 
2494


In [6]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [7]:
x_train.shape

(25000,)

In [8]:
y_train.shape

(25000,)

In [9]:
x_test.shape

(25000,)

In [10]:
y_test.shape

(25000,)

In [11]:
word_index = imdb.get_word_index()

reverse_word_index = dict(
[(value, key) for (key, value) in word_index.items()])

decoded_review = ' '.join(
[reverse_word_index.get(i - 3, '?') for i in x_train[123]])

print(decoded_review)

1646592/1641221 [==============================] - 1s 0us/step
? beautiful and touching movie rich colors great settings good acting and one of the most charming movies i have seen in a while i never saw such an interesting setting when i was in china my wife liked it so much she asked me to ? on and rate it so other would enjoy too


In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [0]:
#load dataset as a list of ints
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [14]:
print("x_train ", x_train.shape)
print("y_train ", y_train.shape)
print("_"*100)
print("x_test ", x_test.shape)
print("y_test ", y_test.shape)
print("_"*100)
print("Maximum value of a word index ")
print(max([max(sequence) for sequence in x_train]))
print("Maximum length num words of review in train ")
print(max([len(sequence) for sequence in x_train]))

x_train  (25000, 300)
y_train  (25000,)
____________________________________________________________________________________________________
x_test  (25000, 300)
y_test  (25000,)
____________________________________________________________________________________________________
Maximum value of a word index 
9999
Maximum length num words of review in train 
300


In [15]:
x_train.shape

(25000, 300)

In [16]:
x_test.shape

(25000, 300)

In [17]:
imdb

<module 'keras.datasets.imdb' from '/usr/local/lib/python3.6/dist-packages/keras/datasets/imdb.py'>

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

W0803 02:10:19.925750 139760945166208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 02:10:19.964774 139760945166208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 02:10:19.970822 139760945166208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 02:10:20.093263 139760945166208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0803 02:10:20.102780 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [19]:
batch_size = 50

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 270s 11ms/step - loss: 0.4654 - acc: 0.7819 - val_loss: 0.4210 - val_acc: 0.8125
Epoch 2/10
25000/25000 [==============================] - 266s 11ms/step - loss: 0.3585 - acc: 0.8499 - val_loss: 0.3649 - val_acc: 0.8425
Epoch 3/10
25000/25000 [==============================] - 268s 11ms/step - loss: 0.3460 - acc: 0.8517 - val_loss: 0.3846 - val_acc: 0.8422
Epoch 4/10
25000/25000 [==============================] - 267s 11ms/step - loss: 0.2971 - acc: 0.8788 - val_loss: 0.3623 - val_acc: 0.8510
Epoch 5/10
25000/25000 [==============================] - 265s 11ms/step - loss: 0.2217 - acc: 0.9152 - val_loss: 0.3541 - val_acc: 0.8637
Epoch 6/10
25000/25000 [==============================] - 269s 11ms/step - loss: 0.1851 - acc: 0.9297 - val_loss: 0.3941 - val_acc: 0.8512
Epoch 7/10
25000/25000 [==============================] - 267s 11ms/step - loss: 0.1496 - acc: 0.9447 - val_loss: 0.

The above model is giving 96.85% accuracy

In [20]:
pwd

'/content/drive/My Drive/Colab Notebooks/LabExternal9'

In [0]:
# save the model to file
model.save('./SeqNLP_Project1_model.h5')

## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [22]:
pwd

'/content/drive/My Drive/Colab Notebooks/LabExternal9'

In [0]:
# load the model
model = load_model('./SeqNLP_Project1_model.h5')

In [24]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 225s 9ms/step - loss: 0.0725 - acc: 0.9746
Epoch 2/10
25000/25000 [==============================] - 286s 11ms/step - loss: 0.0644 - acc: 0.9783
Epoch 3/10
25000/25000 [==============================] - 242s 10ms/step - loss: 0.0416 - acc: 0.9859
Epoch 4/10
25000/25000 [==============================] - 235s 9ms/step - loss: 0.0368 - acc: 0.9893
Epoch 5/10
25000/25000 [==============================] - 226s 9ms/step - loss: 0.0364 - acc: 0.9880
Epoch 6/10
25000/25000 [==============================] - 226s 9ms/step - loss: 0.0356 - acc: 0.9879
Epoch 7/10
25000/25000 [==============================] - 227s 9ms/step - loss: 0.0303 - acc: 0.9903
Epoch 8/10
25000/25000 [==============================] - 228s 9ms/step - loss: 0.0258 - acc: 0.9917
Epoch 9/10
25000/25000 [==============================] - 227s 9ms/step - loss: 0.0195 - acc: 0.9936
Epoch 10/10
25000/25000 [==============================] - 223s 9ms/step - loss: 0.0242 -

In [25]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 45s 2ms/step
Test score: 0.7888527420796454
Test accuracy: 0.8492799950838089
